## Gensim LDA for topic extraction

In [1]:
import spacy
import pandas as pd

#vis
import pyLDAvis
import pyLDAvis.gensim_models

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from spacy.lang.nl import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

c:\_projects\KPMG_project\venvkpmg\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [2]:
stopwords=STOP_WORDS

In [3]:
df=pd.read_csv("data_03.csv",sep="|")
data=df['nl_sum']


In [4]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("nl_core_news_lg")
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

c:\_projects\KPMG_project\venvkpmg\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\_projects\KPMG_project\venvkpmg\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\_projects\KPMG_project\venvkpmg\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\_projects\KPMG_project\venvkpmg\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\_projects\KPMG_project\venvkpmg\lib\s

demissionair minister Brussels hoofdstedelijk regering belasten Financiën begroting leggen


In [5]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['demissionair', 'minister', 'brussels', 'hoofdstedelijk', 'regering', 'belasten', 'financien', 'begroting', 'leggen', 'voorwaarde', 'behoud', 'fiscaal', 'gunstregime', 'bekomen', 'kader', 'overdracht', 'familiaal', 'onderneming', 'familiaal', 'vennootschap']


In [6]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1)]
aanving


In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           iterations=150,
                                           alpha="auto")


In [8]:
lda_model.print_topics()


[(0,
  '0.139*"wijzigen" + 0.099*"toekenning" + 0.099*"woord" + 0.083*"onderwijs" + 0.062*"vervangen" + 0.053*"overeenkomst" + 0.036*"aanpassing" + 0.032*"rekening" + 0.022*"ook" + 0.022*"kracht"'),
 (1,
  '0.099*"dienst" + 0.068*"komen" + 0.050*"overheidsdienst" + 0.043*"inkomen" + 0.042*"coordinatie" + 0.039*"federale" + 0.029*"lezen" + 0.025*"aan_wijzen" + 0.022*"bis" + 0.015*"kadastraal"'),
 (2,
  '0.145*"volksgezondheid" + 0.090*"ministerie" + 0.086*"veiligheid" + 0.079*"justitie" + 0.067*"demissionair" + 0.066*"minister" + 0.056*"welzijn" + 0.024*"plaats" + 0.020*"daartoe" + 0.018*"voorlopig"'),
 (3,
  '0.296*"definitief" + 0.098*"besluit" + 0.095*"ministerieel" + 0.063*"wijziging" + 0.059*"beperken" + 0.055*"maatregel" + 0.054*"verspreiding" + 0.047*"coronavirus" + 0.040*"houd_en" + 0.031*"dringend"'),
 (4,
  '0.119*"financien" + 0.102*"ministerie" + 0.097*"nieuw" + 0.083*"minister" + 0.070*"in_dienen" + 0.050*"niet" + 0.032*"houdenden" + 0.031*"macht" + 0.027*"toe_kennen" + 0.0

In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds="mmds", R=30)
vis

c:\_projects\KPMG_project\venvkpmg\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.458204 -0.143035       1        1  19.325892
3     -0.452045  0.101432       2        1   9.942848
6     -0.243366 -0.368871       3        1   9.638511
4      0.081802 -0.444502       4        1   8.665764
14     0.018719  0.459598       5        1   8.593274
8     -0.152700  0.355996       6        1   4.851868
12    -0.244408  0.203127       7        1   4.069729
13    -0.218276 -0.147700       8        1   4.062263
17     0.032793 -0.295912       9        1   4.015527
5      0.382571 -0.080807      10        1   3.950838
2      0.276756 -0.271536      11        1   3.938304
0      0.330342  0.227963      12        1   3.490938
9     -0.045437 -0.142672      13        1   2.756971
19    -0.145710  0.062684      14        1   2.633110
11     0.013207  0.196503      15        1   2.319981
10     0.164093  0.235811      16        1   1.987044
16     0.145717 -0.104813      17        1   1.928096
7      0.275491  0.053631      18        1   1.490862
1      0.158960  0.024464      19        1   1.215949
15     0.079693  0.078637      20        1   1.122233, topic_info=                Term        Freq       Total Category  logprob  loglift
198       definitief  386.000000  386.000000  Default  30.0000  30.0000
7            besluit  484.000000  484.000000  Default  29.0000  29.0000
28          regering  240.000000  240.000000  Default  28.0000  28.0000
60            hebben  186.000000  186.000000  Default  27.0000  27.0000
332        maatregel  201.000000  201.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
1751          intern    0.035842    1.049376  Topic20  -8.3168   1.1130
1119   overeen_komen    0.035840    1.049330  Topic20  -8.3168   1.1130
1060        voorstel    0.035845    1.049481  Topic20  -8.3167   1.1130
1542  basisallocatie    0.035842    1.049422  Topic20  -8.3168   1.1130
1544       programma    0.035842    1.049491  Topic20  -8.3168   1.1129

[691 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
121       3  0.938256   aan_brengen
273      13  0.614536   aan_dringen
482       2  0.935624  aan_kondigen
56        9  0.991160     aan_nemen
287       4  0.965996    aan_pakken
...     ...       ...           ...
120      15  0.854260          zien
247      16  0.950095            zo
606       5  0.781153          zorg
367      11  0.740587  zorgverlener
622       5  0.559321         zover

[542 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 4, 7, 5, 15, 9, 13, 14, 18, 6, 3, 1, 10, 20, 12, 11, 17, 8, 2, 16])